[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2020CSharp/main?urlpath=lab%2Ftree%2FDay08.ipynb)

# --- Day 8: Handheld Halting ---

In [1]:
using System.IO;

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
enum Operations
{
    acc,
    jmp,
    nop,
}

In [3]:
record Instruction(Operations Operation, int Argument);

In [4]:
var bootCode = File
    .ReadAllLines(@"input/08.txt")
    .Select(line => line.Split(" "))
    .Select(arr => new Instruction((Operations)Enum.Parse<Operations>(arr.First()), Convert.ToInt32(arr.Last())))
    .ToArray();

In [5]:
(int Accumulator, int Pointer) RunBootCode(Instruction[] instructions)
{
    var accumulator = 0;
    var pointer = 0;
    bool[] visited = new bool[instructions.Length];
    
    while (pointer < instructions.Length && !visited[pointer])
    {
        var ins = instructions[pointer];
        visited[pointer] = true;
        
        switch (ins.Operation)
        {
            case Operations.acc:
                accumulator += ins.Argument;
                pointer++;
                break;
            case Operations.jmp:
                pointer += ins.Argument;
                break;
            case Operations.nop:
                pointer++;
                break;
        }
    }
    
    return (accumulator, pointer);
}

In [6]:
RunBootCode(bootCode).Accumulator

1610

# --- Part Two ---

In [7]:
int FixBootCode(Instruction[] instructions)
{
    for (int i = 0; i < instructions.Length; i++)
    {
        var acc = 0;
        var ptr = 0;
        var ins = instructions[i];
        
        switch (ins.Operation)
        {
            case Operations.jmp:
                (acc, ptr) = RunBootCode(instructions.Take(i)
                    .Append(new Instruction(Operations.nop, ins.Argument))
                    .Concat(instructions.Skip(i + 1)).ToArray()
                );
                break;
            case Operations.nop:
                (acc, ptr) = RunBootCode(instructions.Take(i)
                    .Append(new Instruction(Operations.jmp, ins.Argument))
                    .Concat(instructions.Skip(i + 1)).ToArray()
                );
                break;
        }

        if (ptr >= instructions.Length)
        {
            return acc;
        }
    }
    return -1;
}

In [8]:
FixBootCode(bootCode)

1703